In [1]:
import csv
import pandas as pd
import numpy as np
import re
import datetime as dt
import time

In [2]:
df = pd.read_csv("Seattle_Police_Department_911_Incident_Response.csv", parse_dates=[0], infer_datetime_format=True)

In [62]:
df['Event Clearance Date']

0          7/17/2010 20:49
1          7/17/2010 20:50
2          7/17/2010 20:55
3          7/17/2010 21:00
4          7/17/2010 21:00
5          7/17/2010 21:02
6          7/17/2010 21:04
7          7/17/2010 21:05
8          7/17/2010 21:09
9          7/17/2010 21:17
10         7/17/2010 21:18
11         7/17/2010 21:28
12         7/17/2010 21:28
13         7/17/2010 21:30
14         7/17/2010 21:31
15         7/17/2010 21:31
16         7/17/2010 21:39
17         7/17/2010 21:41
18         7/17/2010 21:46
19         7/17/2010 21:47
20         7/17/2010 21:54
21         7/17/2010 21:37
22         7/17/2010 21:41
23         7/17/2010 21:46
24         7/17/2010 21:48
25         7/17/2010 20:33
26         7/17/2010 20:33
27         7/17/2010 20:38
28         7/17/2010 20:39
29         7/17/2010 20:42
                ...       
1048545    5/12/2014 18:01
1048546    5/12/2014 18:09
1048547    5/12/2014 18:07
1048548    5/12/2014 18:06
1048549                NaN
1048550    5/12/2014 18:10
1

In [70]:
df['fullDate'] = df['Event Clearance Date'].apply(lambda x: str(x).split(' ')[0])
df

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,Zone/Beat,Census Tract,Longitude,Latitude,Incident Location,Initial Type Description,Initial Type Subgroup,Initial Type Group,At Scene Time,fullDate
0,﻿15736,10000246357,2010246357,242.0,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,7/17/2010 20:49,3XX BLOCK OF PINE ST,M,M2,8100.2001,-122.338147,47.610975,"(47.610975163, -122.338146748)",NaN,NaN,NaN,NaN,7/17/2010
1,15737,10000246471,2010246471,65.0,THEFT - MISCELLANEOUS,THEFT,OTHER PROPERTY,7/17/2010 20:50,36XX BLOCK OF DISCOVERY PARK BLVD,Q,Q1,5700.1012,-122.404613,47.658325,"(47.658324899, -122.404612874)",NaN,NaN,NaN,NaN,7/17/2010
2,15738,10000246255,2010246255,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",7/17/2010 20:55,21XX BLOCK OF 3RD AVE,M,M2,7200.2025,-122.342843,47.613551,"(47.613551471, -122.342843234)",NaN,NaN,NaN,NaN,7/17/2010
3,15739,10000246473,2010246473,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,7/17/2010 21:00,7XX BLOCK OF ROY ST,D,D1,7200.1002,-122.341847,47.625401,"(47.625401388, -122.341846999)",NaN,NaN,NaN,NaN,7/17/2010
4,15740,10000246330,2010246330,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",7/17/2010 21:00,9XX BLOCK OF ALOHA ST,D,D1,6700.1009,-122.339709,47.627425,"(47.627424837, -122.339708605)",NaN,NaN,NaN,NaN,7/17/2010
5,15741,10000246477,2010246477,281.0,SUSPICIOUS VEHICLE,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,7/17/2010 21:02,30XX BLOCK OF W GOVERNMENT WAY,Q,Q1,5700.2005,-122.396627,47.661312,"(47.66131158, -122.39662681)",NaN,NaN,NaN,NaN,7/17/2010
6,15742,10000246436,2010246436,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,7/17/2010 21:04,50XX BLOCK OF 57TH AVE S,R,R3,10200.1012,-122.263332,47.555956,"(47.555955852, -122.263332426)",NaN,NaN,NaN,NaN,7/17/2010
7,15743,10000246472,2010246472,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,7/17/2010 21:05,56XX BLOCK OF BROOKLYN AVE NE,U,U3,4400.5003,-122.314166,47.670345,"(47.670345345, -122.3141656)",NaN,NaN,NaN,NaN,7/17/2010
8,15744,10000246493,2010246493,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,7/17/2010 21:09,15XX BLOCK OF 4TH AVE,K,K1,8100.2005,-122.336891,47.610440,"(47.610440245, -122.336890515)",NaN,NaN,NaN,NaN,7/17/2010
9,15745,10000246478,2010246478,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,7/17/2010 21:17,100XX BLOCK OF COLLEGE WAY N,N,N3,1300.1009,-122.334524,47.702332,"(47.702331629, -122.33452433)",NaN,NaN,NaN,NaN,7/17/2010


In [16]:
weather = pd.read_csv("WEATHER.csv")

In [17]:
weather

,﻿year,month,date,tempHigh,tempAvg,tempLow,dewHigh,dewAvg,dewLow,humidityHigh,...,seaLvlAvg,seaLvlLow,visibilityHigh,visibilityAvg,visibilityLow,windHigh,windAvg,windHigh.1,precip,events
0,2015,Jan,1,43,35,27,28,25,21,81,...,30.31,30.22,10,10,6,4,0,-,0,
1,2015,Jan,2,44,38,32,39,32,25,86,...,30.16,30.10,10,10,7,12,4,-,0.03,
2,2015,Jan,3,43,38,33,38,35,30,93,...,30.26,30.22,10,7,3,7,2,-,0,Rain
3,2015,Jan,4,54,48,41,48,41,37,89,...,30.14,29.98,10,8,3,21,7,32,0.22,Rain
4,2015,Jan,5,57,56,54,52,50,48,88,...,30.18,30.00,10,8,2,20,14,31,0.07,Rain
5,2015,Jan,6,55,50,46,48,46,43,93,...,30.29,30.26,10,7,0,6,2,-,0.01,Fog
6,2015,Jan,7,50,47,44,46,43,41,93,...,30.28,30.25,10,5,1,6,2,-,0,
7,2015,Jan,8,48,44,39,41,38,36,89,...,30.17,30.12,10,6,0,8,3,-,0,Fog
8,2015,Jan,9,50,44,39,41,39,37,93,...,30.09,30.02,10,4,0,6,2,-,0.01,"Fog , Rain"
9,2015,Jan,10,48,46,44,45,44,41,93,...,30.03,30.00,10,3,1,7,1,-,0.18,Rain


In [19]:
import calendar
weather.shape
list(weather.columns.values)

['\ufeffyear',
 'month',
 'date',
 'tempHigh',
 'tempAvg',
 'tempLow',
 'dewHigh',
 'dewAvg',
 'dewLow',
 'humidityHigh',
 'humidityAvg',
 'humidityLow',
 'seaLvlHigh',
 'seaLvlAvg',
 'seaLvlLow',
 'visibilityHigh',
 'visibilityAvg',
 'visibilityLow',
 'windHigh',
 'windAvg',
 'windHigh.1',
 'precip',
 'events']

In [24]:
months_map = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr':'4', 'May': '5', 'Jun': '6', 'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct':'10', 'Nov': '11', 'Dec': '12'}

In [26]:
weather['m'] = weather['month'].apply(lambda x: months_map[x])

In [28]:
weather

,﻿year,month,date,tempHigh,tempAvg,tempLow,dewHigh,dewAvg,dewLow,humidityHigh,...,seaLvlLow,visibilityHigh,visibilityAvg,visibilityLow,windHigh,windAvg,windHigh.1,precip,events,m
0,2015,Jan,1,43,35,27,28,25,21,81,...,30.22,10,10,6,4,0,-,0,,1
1,2015,Jan,2,44,38,32,39,32,25,86,...,30.10,10,10,7,12,4,-,0.03,,1
2,2015,Jan,3,43,38,33,38,35,30,93,...,30.22,10,7,3,7,2,-,0,Rain,1
3,2015,Jan,4,54,48,41,48,41,37,89,...,29.98,10,8,3,21,7,32,0.22,Rain,1
4,2015,Jan,5,57,56,54,52,50,48,88,...,30.00,10,8,2,20,14,31,0.07,Rain,1
5,2015,Jan,6,55,50,46,48,46,43,93,...,30.26,10,7,0,6,2,-,0.01,Fog,1
6,2015,Jan,7,50,47,44,46,43,41,93,...,30.25,10,5,1,6,2,-,0,,1
7,2015,Jan,8,48,44,39,41,38,36,89,...,30.12,10,6,0,8,3,-,0,Fog,1
8,2015,Jan,9,50,44,39,41,39,37,93,...,30.02,10,4,0,6,2,-,0.01,"Fog , Rain",1
9,2015,Jan,10,48,46,44,45,44,41,93,...,30.00,10,3,1,7,1,-,0.18,Rain,1


In [126]:
weather['fullDate'] = weather.apply(lambda x: str(x['m']) + '/' + str(x['date']) + '/' + str(x['\ufeffyear']), axis=1 )
#If the apply doesn't work you may have to pass 'axis=1' after the lambda
#(pass in df['weather'] or whatever your column is called)
weather['clear'] = weather['events'].apply(lambda x: 1 if ('Rain' in str(x) or 'Snow' in str(x) or 'Fog' in str(x) or 'Thunderstorm' in str(x)) else 0)
weather['notClear'] = weather['clear'].apply(lambda x: 1 if x is 0 else 0)

In [127]:
weather[200:]

,﻿year,month,date,tempHigh,tempAvg,tempLow,dewHigh,dewAvg,dewLow,humidityHigh,...,visibilityLow,windHigh,windAvg,windHigh.1,precip,events,m,fullDate,clear,notClear
200,2015,Jul,20,82,74,66,58,54,53,75,...,10,13,8,21,0,,7,7/20/2015,0,1
201,2015,Jul,21,75,68,62,56,52,49,78,...,4,9,3,-,0.13,Rain,7,7/21/2015,1,0
202,2015,Jul,22,77,68,61,52,50,48,70,...,10,10,4,-,0,,7,7/22/2015,0,1
203,2015,Jul,23,81,71,62,51,50,48,65,...,10,7,2,-,0,,7,7/23/2015,0,1
204,2015,Jul,24,77,67,59,59,53,50,76,...,7,13,6,24,0.01,Rain,7,7/24/2015,1,0
205,2015,Jul,25,70,66,62,61,54,49,88,...,5,12,4,16,0.02,Rain,7,7/25/2015,1,0
206,2015,Jul,26,73,66,60,56,52,49,80,...,4,8,2,-,0.1,Rain,7,7/26/2015,1,0
207,2015,Jul,27,77,65,55,54,51,45,84,...,6,7,2,-,0.01,,7,7/27/2015,0,1
208,2015,Jul,28,82,72,61,53,51,47,72,...,10,9,3,-,0,,7,7/28/2015,0,1
209,2015,Jul,29,89,75,62,55,51,48,75,...,9,10,3,18,0,,7,7/29/2015,0,1


In [129]:
list(df.columns.values)

['CAD CDW ID',
 'CAD Event Number',
 'General Offense Number',
 'Event Clearance Code',
 'Event Clearance Description',
 'Event Clearance SubGroup',
 'Event Clearance Group',
 'Event Clearance Date',
 'Hundred Block Location',
 'District/Sector',
 'Zone/Beat',
 'Census Tract',
 'Longitude',
 'Latitude',
 'Incident Location',
 'Initial Type Description',
 'Initial Type Subgroup',
 'Initial Type Group',
 'At Scene Time',
 'fullDate']

In [128]:
weather.events.unique()

array([' ', 'Rain', 'Fog', 'Fog , Rain', 'Rain , Thunderstorm', 'Snow',
       'Rain , Snow', 'Fog , Rain , Snow', nan, 'Fog , Snow',
       'Fog , Rain , Thunderstorm', 'Rain , Snow , Thunderstorm',
       'Thunderstorm'], dtype=object)

In [130]:
merged = pd.merge(df, weather, on = 'fullDate', how = 'left')

In [140]:
merged

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,visibilityAvg,visibilityLow,windHigh,windAvg,windHigh.1,precip,events,m,clear,notClear
0,﻿15736,10000246357,2010246357,242.0,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,7/17/2010 20:49,3XX BLOCK OF PINE ST,M,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
1,15737,10000246471,2010246471,65.0,THEFT - MISCELLANEOUS,THEFT,OTHER PROPERTY,7/17/2010 20:50,36XX BLOCK OF DISCOVERY PARK BLVD,Q,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
2,15738,10000246255,2010246255,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",7/17/2010 20:55,21XX BLOCK OF 3RD AVE,M,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
3,15739,10000246473,2010246473,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,7/17/2010 21:00,7XX BLOCK OF ROY ST,D,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
4,15740,10000246330,2010246330,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",7/17/2010 21:00,9XX BLOCK OF ALOHA ST,D,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
5,15741,10000246477,2010246477,281.0,SUSPICIOUS VEHICLE,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,7/17/2010 21:02,30XX BLOCK OF W GOVERNMENT WAY,Q,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
6,15742,10000246436,2010246436,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,7/17/2010 21:04,50XX BLOCK OF 57TH AVE S,R,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
7,15743,10000246472,2010246472,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,7/17/2010 21:05,56XX BLOCK OF BROOKLYN AVE NE,U,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
8,15744,10000246493,2010246493,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,7/17/2010 21:09,15XX BLOCK OF 4TH AVE,K,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
9,15745,10000246478,2010246478,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,7/17/2010 21:17,100XX BLOCK OF COLLEGE WAY N,N,...,10.0,10.0,10.0,6.0,14,0,NaN,7,0.0,1.0
